<a href="https://colab.research.google.com/github/RyanChen12035/capstone/blob/main/two_towers_data_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Conventional two tower strucutre is for recommanding system and it takes in structured data on the query side (clicking or not, staying time..et al.) and retrieve structured data on the candidate side (recommanded youtube channel, tracks on spotify). They use DNN on both sides and train the two tower model so they are sharing a same embedding space.

In this case, we are aiming for taking in structured data such as prompt here, and retrieve confidential data in the candidate tower. As a result, we meed a BERT model and its embedding to power the vector database. We plan to train a BERT model and set it as the main body of both towers. Extract the embedding from the last layer of the Transformer in the BERT and use it to power FAISS vector database.

Part 1 Data preparation: Set job title as our blacklist. Mix data of job description and medical diagnosis each with 0 and 1 tagging.
Encapsulate it with torch dataset and dataloader, split and shuffle

Part 2 Model and tokenizer: training a text classification model.

Part 3 Extract the embedding space of BERT model and use it to power the vector database.

In [1]:
!pip install streamlit -q
!pip install PyPDF2 -q
!pip install langchain -q
!pip install sentence-transformers -q
!pip install faiss-cpu
!pip install faiss-gpu
!pip install pydot --quiet
!pip install transformers --quiet
!pip install datasets --quiet
!wget -q -O - ipv4.icanhazip.com

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 940.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.6/252.6 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install datasets --quiet

In [ ]:
!wget -q -O - ipv4.icanhazip.com

In [ ]:
!huggingface-cli login

In [3]:
import numpy as np
import pandas as pd
from datasets import load_dataset

In [ ]:
!pwd

/content


In [186]:
# control + enter ten times
df = pd.read_csv("./fake_job_postings.csv", error_bad_lines=False)
df_job_blend = df
df.head(5)

<ipython-input-186-f219c6808362>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv("./fake_job_postings.csv", error_bad_lines=False)


,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [203]:
def concatenate_cols(row):
    if pd.isna(row['department']):
        row['department'] = 'freelancing'
    if pd.isna(row['salary_range']):
        row['salary_range'] = 'negotiable'
    if pd.isna(row['benefits']):
        row['benefits'] = 'negotiable'
    if len(str(row['company_profile'])) > 150:
        row['company_profile'] = row['company_profile'][:150]
    if len(str(row['description'])) > 150:
        row['description'] = row['description'][:150]
    if len(str(row['requirements'])) > 150:
        row['requirements'] = row['requirements'][:150]

    # data augmentation
    if row['job_id'] % 5 == 0:
        job_openingline = f"{row['title']} and the salary range is {row['salary_range']} "
        job_description = f"{row['company_profile']}"
    elif row['job_id'] % 5 == 1:
        job_openingline = f"I am doing {row['title']} at {row['location']} "
        job_description = f"{row['description']}. {row['requirements']}, also the benefit is great, benefit's {row['benefits']}"
    elif row['job_id'] % 5 == 2:
        job_openingline = f"I am doing in the industry of {row['department']} "
        job_description = f"{row['requirements']}"
    elif row['job_id'] % 5 == 3:
        job_openingline = f"My company is at {row['location']} and they payroll is around {row['salary_range']} "
        job_description = f"{row['company_profile']}, {row['description']}"
    else:
        job_openingline = f" "
        job_description = f"{row['company_profile']}, {row['description']}, and the benefits: {row['benefits']}"

    return job_openingline + job_description

df['prompt'] = df.apply(lambda row: concatenate_cols(row), axis=1)
df['related_to_job'] = 1
df = df[['prompt', 'related_to_job']]
df.head(10)

,prompt,related_to_job
0,"I am doing Marketing Intern at US, NY, New Yor...",1
1,I am doing in the industry of Success What we ...,1
2,"My company is at US, IA, Wever and they payrol...",1
3,Our passion for improving quality of life thr...,1
4,Bill Review Manager and the salary range is ne...,1
5,"I am doing Accounting Clerk at US, MD, Job Ov...",1
6,I am doing in the industry of ANDROIDPIT Your ...,1
7,"My company is at US, CA, San Francisco and the...",1
8,Solutions3 is a woman-owned small business wh...,1
9,Customer Service Associate - Part Time and th...,1


In [ ]:
df['prompt'][3]

' Our passion for improving quality of life through geography is at the heart of everything we do.\xa0 Esri’s geographic information system (GIS) technolog, THE COMPANY: ESRI – Environmental Systems Research InstituteOur passion for improving quality of life through geography is at the heart of everything , and the benefits: Our culture is anything but corporate—we have a collaborative, creative environment; phone directories organized by first name; a relaxed dress code; and open-door policies.A Place to ThrivePassionate people who strive to make a differenceCasual dress codeFlexible work schedulesSupport for continuing educationCollege-Like CampusA network of buildings amid lush landscaping and numerous outdoor patio areasOn-site café including a Starbucks coffee bar and lounge areaFitness center available 24/7Comprehensive reference library and GIS bibliographyState-of-the-art conference center to host staff and guest speakers\xa0Green InitiativesSolar rooftop panels reduce carbon e

In [43]:
# download diagnosis data from hugging face and mix them together
# Load a dataset by name from the Hugging Face Hub
# https://huggingface.co/datasets/gretelai/symptom_to_diagnosis
symptom_to_diagnosis = load_dataset("gretelai/symptom_to_diagnosis")
symptom_to_diagnosis = symptom_to_diagnosis['train']
df_diagnosis = symptom_to_diagnosis.to_pandas()
df_diagnosis['related_to_job'] = 0
df_diagnosis = df_diagnosis.rename(columns={'input_text': 'prompt'})
df_diagnosis = df_diagnosis[['prompt', 'related_to_job']]
df_diagnosis.head(5)

,prompt,related_to_job
0,I've been having a lot of pain in my neck and ...,0
1,I have a rash on my face that is getting worse...,0
2,I have been urinating blood. I sometimes feel ...,0
3,I have been having trouble with my muscles and...,0
4,I have been feeling really sick. My body hurts...,0


In [170]:
# inject job information into diagnosis
df_diagnosis_blend = symptom_to_diagnosis.to_pandas()
df_dia_job = pd.concat([df_job_blend.iloc[:400], df_diagnosis_blend.iloc[:400]], axis=1)
df_dia_job.head(2)

def df_blend(row):
    if row['job_id'] % 5 == 0:
        diagnosis = row['input_text'][:200]
        job_description = f"{row['title']} and the salary range is {row['salary_range']} "
        return diagnosis + job_description
    elif row['job_id'] % 5 == 1:
        diagnosis = row['input_text'][:200]
        job_description = f"I am doing {row['title']} at {row['location']} "
        return diagnosis + job_description
    elif row['job_id'] % 5 == 2:
        diagnosis = row['input_text'][:200]
        job_description = f"I am doing {row['requirements']} "
        return job_description + diagnosis
    elif row['job_id'] % 5 == 3:
        diagnosis = row['input_text'][:200]
        job_description = f"I am doing {row['requirements']} "
        job_description_2 = f"by the way, the salary range is around {row['salary_range']} "
        return job_description + diagnosis + job_description_2
    elif row['job_id'] % 5 == 4:
        diagnosis = row['input_text'][:200] + ". "
        job_description = f"{row['company_profile']}, {row['description']}"
        return job_description + diagnosis

df_dia_job['prompt'] = df_dia_job.apply(lambda row: df_blend(row), axis=1)
df_dia_job['related_to_job'] = 1
df_dia_job_blend = df_dia_job[['prompt', 'related_to_job']]
df_dia_job_blend.head(2)

,prompt,related_to_job
0,I've been having a lot of pain in my neck and ...,1
1,I am doing What we expect from you:Your key re...,1


In [172]:
print(df_dia_job_blend['prompt'][0])

I've been having a lot of pain in my neck and back. I've also been having trouble with my balance and coordination. I've been coughing a lot and my limbs feel weak.I am doing Marketing Intern at US, NY, New York 


In [177]:
# combine PII by Gemini which is irrelevant to medical diagnosis and diagnosis data. It's better to use medical + PII
# https://www.kaggle.com/code/newtonbaba12345/pii-detection-data-generation-using-gemini/notebook
df_PII = pd.read_csv("./pii_gemini.csv", error_bad_lines=False)
df_PII_blend = df_PII
df_PII.head(5)

<ipython-input-177-e336b766bb90>:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_PII = pd.read_csv("./pii_gemini.csv", error_bad_lines=False)


,Unnamed: 0,Essay,EMAIL,USERNAME,ID_NUM,PHONE_NUM,URL_PERSONAL,STREET_ADDRESS
0,1,"As an essay writer, I am tasked with the respo...","['udavis@hotmail.com', 'jrodriguez@yahoo.com']",['kelleylisa'],[],['+1-839-789-4719'],['https://facebook.com/kellylopez'],"['1122 Megan Squares Suite 848\nPort Jason, TX..."
1,3,"Tyler Lopez, a diligent student with the usern...","['mendozajessica@gmail.com', 'stephenschristin...",['walkerdeborah'],['LNFB87091634579230'],"['+1-622-758-4197x20769', '(942)780-7150x8423']","['https://github.com/emily99', 'https://twitte...","['523 Dana Lane\nJohnsonshire, MT 87296', '142..."
2,5,"Education is the key to a brighter future, and...",['ihays@yahoo.com'],['woodsjohn'],"['146JFQ', '1JN1877']",['351-371-7604x52296'],['https://instagram.com/gonzalezjeffrey'],"['8847 Kramer Station\nSouth Christopher, IN 3..."
3,6,"In today's modern era, the internet has become...",['leachshannon@hotmail.com'],[],"['GDPM43039213765821', 'GB63TPES66875773893055...",['8402752275'],['https://facebook.com/gregorylinda'],"['634 Petersen Orchard\nLawsonbury, PR 03185']"
4,7,My academic journey has been a tapestry woven ...,['pbrown@gmail.com'],['justingreer'],"['057-05-7887', '79-1256E']","['2972670172', '+1-992-751-8536x710']",['https://github.com/montesjennifer'],"['4183 Kimberly Mills\nKatherinemouth, MI 83340']"


In [200]:
df_PII['related_to_job'] = 0
df_PII['prompt'] = df_PII['Essay']
df_PII = df_PII[['prompt', 'related_to_job']]
df_PII.head(5)
print(df_PII.shape[0])

400


<ipython-input-200-d8c5890da7e6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_PII['related_to_job'] = 0
<ipython-input-200-d8c5890da7e6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_PII['prompt'] = df_PII['Essay']


In [184]:
df_PII_blend['Essay'][0:2]

0    As an essay writer, I am tasked with the respo...
1    Tyler Lopez, a diligent student with the usern...
Name: Essay, dtype: object

In [197]:
# inject job information into PII
df_job_PII_blend = pd.concat([df_job_blend.iloc[:400], df_PII_blend['Essay'][:400]], axis=1)
df_job_PII_blend.head(2)

def job_PII_blend(row):
    if row['job_id'] % 5 == 0:
        PII = row['Essay'][:200]
        job_description = f"{row['title']} and the salary range is {row['salary_range']} "
        return PII + job_description
    elif row['job_id'] % 5 == 1:
        PII = row['Essay'][:230]
        job_description = f"I am doing {row['title']} at {row['location']} "
        return PII + job_description
    elif row['job_id'] % 5 == 2:
        PII = row['Essay'][:250]
        job_description = f"I am doing {row['requirements']} "
        return job_description + PII
    elif row['job_id'] % 5 == 3:
        PII = row['Essay'][:260]
        job_description = f"I am doing {row['requirements']} "
        job_description_2 = f"by the way, the salary range is around {row['salary_range']} "
        return job_description + PII + job_description_2
    elif row['job_id'] % 5 == 4:
        PII = row['Essay'][:270] + ". "
        job_description = f"{row['company_profile']}, {row['description']}"
        return job_description + PII

df_job_PII_blend['prompt'] = df_job_PII_blend.apply(lambda row: job_PII_blend(row), axis=1)
df_job_PII_blend['related_to_job'] = 1
df_job_PII_blend = df_job_PII_blend[['prompt', 'related_to_job']]
df_job_PII_blend.head(2)

,prompt,related_to_job
0,"As an essay writer, I am tasked with the respo...",1
1,I am doing What we expect from you:Your key re...,1


In [205]:
print(f"number of job_df {df.shape[0]}")
print(f"number of diagnosis_df {df_diagnosis.shape[0]}")
print(f"number of PII_df {df_PII.shape[0]}")
print(f"number of PII_df {df_dia_job_blend.shape[0]}")
print(f"number of PII_df {df_job_PII_blend.shape[0]}")

# in case of imbalanced data
df_job = df.iloc[:400]
df_PII = df_PII.iloc[:400]
df_diagnosis = df_diagnosis.iloc[:400]

df_final = pd.concat([df_job, df_diagnosis, df_PII, df_dia_job_blend, df_job_PII_blend])
print(df_final.shape[0])
df_final.head(5)

number of job_df 17880
number of diagnosis_df 400
number of PII_df 400
number of PII_df 400
number of PII_df 400
2000


,prompt,related_to_job
0,"I am doing Marketing Intern at US, NY, New Yor...",1
1,I am doing in the industry of Success What we ...,1
2,"My company is at US, IA, Wever and they payrol...",1
3,Our passion for improving quality of life thr...,1
4,Bill Review Manager and the salary range is ne...,1


In [207]:
df_final[df_final['related_to_job'] == 0]

,prompt,related_to_job
0,I've been having a lot of pain in my neck and ...,0
1,I have a rash on my face that is getting worse...,0
2,I have been urinating blood. I sometimes feel ...,0
3,I have been having trouble with my muscles and...,0
4,I have been feeling really sick. My body hurts...,0
...,...,...
395,"In the vibrant tapestry of human existence, wh...",0
396,"In the realm of education, students embark on ...",0
397,**The Journey of Lauren Dixon: Embracing the P...,0
398,**Academic Excellence: A Pathway to Personal F...,0


In [208]:
df_final.to_csv('job_PII_diagnosis_3.csv', index=False)